### Code: Berezinskii-Kosterlitz-Thouless Renormalization Group Flow for Finite Size Scaling of the Bose-Hubbard Model
Authors


#### Abstract 
abstract

#### Requirements 
To run this code, Julia version 1.10.4 or higher is required. The cell below will install the required packages.

### Install packages

In [ ]:
using Pkg 
Pkg.activate(".")
Pkg.add([
    "Plots", 
    "PyFormattedStrings", 
    "NonlinearSolve", 
    "StaticArrays", 
    "Printf", 
    "Integrals", 
    "FastClosures", 
    "LaTeXStrings", 
    "DataFrames",  
    "NPZ", 
    "Measures", 
    "PyCall"]);

### Import

In [ ]:
using Plots
using PyFormattedStrings
using NonlinearSolve
using StaticArrays
using Printf
using Integrals
using FastClosures
using LaTeXStrings
using DataFrames 
using NPZ
using Measures
using PyCall

### Include 

In [ ]:
include("./plot_utils.jl"); 
include("./load.jl");
include("./fit.jl");
include("./rg.jl");
include("./fss.jl")
include("./plot_routines.jl");

### Figure 1: BKT RG Flow
RG flow close to the BKT phase transition constructed from the DMRG simulations via 
 
\begin{align*}
    2\ln\!\left(\frac{L_2}{L_1}\right)  &= \int_{K_2}^{K_1} \frac{{\rm d}K}{K^2 (\ln(K/2)-\zeta) + 2K} \\
            \zeta  &\equiv \frac{2}{K} + \ln\left(\frac{K}{2}\right) - \frac{\pi^2}{32} \left(\frac{V}{E_r}\right)^2  \ .
\end{align*}
 
Each line corresponds to one value of $\zeta(U)$ obtained from the largest two system sizes for each $U/J$. Here, $K$ is the Luttinger parameter and $V$ the strength of the cosine lattice potential and $E_r$ a characteristic energy scale. Circles show the points for individual system sizes (larger circles correspond to larger $L$, $L\in\{16,32,48,64,96,128\}$). The dashed black line is the separatrix marking the phase transition $\zeta(U_c)=1$. Open circles are obtained by fits of the $L\to\infty$ limit of 
 
\begin{align*}
    \mathcal{F}_{\rm{pbc}}(\ell; K, \alpha) &= 
    \frac{K}{2\pi^2}\ln{\left[1+\frac{\sin^2{(\pi\ell/L)}}{\sinh^2{(\pi\alpha/L)}}\right]}   
\end{align*}
 
to data obtained for infinite boundary conditions using the VUMPS algorithm [V. Zauner-Stauber, L. Vanderstraeten, M. T. Fishman,F. Verstraete, and J. Haegeman, Variational optimization algorithms for uniform matrix product states, Physical Review B 97, 045145 (2018)]. Crosses are obtained from QMC. We find excellent agreement of the DMRG data with the predicted RG flow and between the different methods.

In [ ]:
begin  
    # color lookup
	Us = get_values_of_U(;path="../data/pbc/DMRG/")
	local col_lookup = get_color_lookup(Us)
	# plot options
	local l = grid(1, 2, widths=[0.8,0.2])
    local ms_lookup =  get_marker_size_lookup()


    # pbc dmrg
	local p1 = plot(; xlabel=L"$K$", ylabel=L"$\pi V/(4E_r)$", ylim=(-0.001,.3999), xlim=(1.645,2.0+.77), legend=nothing,foreground_color_legend =nothing, legendtitle = nothing, legendtitlefontsize=9, handlelength=8  ) 
    local Us, ζs, ζserr = plot_dmrg_rg_flow!(p1, col_lookup, ms_lookup ) 
	# avoid accidental overwrite for values required below
	__Us, __ζs, __ζserr = Us, ζs, ζserr
	# separatrix 
    local Ks_sep, Vs_sep = get_flow_curve(1.0)
    plot!(p1, Ks_sep[Ks_sep.>=2.0], Vs_sep[Ks_sep.>=2.0]; color=:black, linewidth=1.0, linestyle=:dash, label=nothing) 
	add_arrow!(p1,Ks_sep[Ks_sep.>2.0], Vs_sep[Ks_sep.>2.0], 0.28,:black;debug_print=true)
	add_arrow!(p1,Ks_sep[Ks_sep.>2.0], Vs_sep[Ks_sep.>2.0], 0.35,:black;debug_print=true)
	plot!(p1, Ks_sep[Ks_sep.<2.0], Vs_sep[Ks_sep.<2.0]; color=:black, linewidth=1.0, linestyle=:dot, label=nothing)
	add_arrow!(p1,Ks_sep[Ks_sep.<2.0], Vs_sep[Ks_sep.<2.0], 0.1,:black;debug_print=true)
	add_arrow!(p1,Ks_sep[Ks_sep.<2.0], Vs_sep[Ks_sep.<2.0], 0.2,:black;debug_print=true)
	add_arrow!(p1,Ks_sep[Ks_sep.<2.0], Vs_sep[Ks_sep.<2.0], 0.3,:black;debug_print=true)

	# vumps data 
	plot_vumps_rg_flow!(p1, col_lookup, ms_lookup )

	# qmc data 
	plot_qmc_rg_flow!(p1, col_lookup, ms_lookup )
	
	# manually draw legend 
	local p_leg = plot(; framestyle = nothing) 
	draw_legend!(p_leg, Us, ζs, col_lookup, ms_lookup, highlight_U = [3.275])	
	
	# add phase labels
	annotate!(p1, 2.0, 0.38, text(L"\textrm{MI}", 10, :vcenter, :hcenter))
	annotate!(p1, 2.625, 0.38, text(L"\textrm{SF}", 10, :vcenter, :hcenter)) 
	
	
	local p = plot(p1, p_leg, layout = l, size=(1.2*600,1.1*600/2), bottom_margin=[2mm 2mm], left_margin=[2mm -5mm] )  
	savefig(p,"../figures/001_BKT_RG_flow_Bose-Hubbard.pdf");
	savefig(p,"../figures/001_BKT_RG_flow_Bose-Hubbard.svg");
	#p 
end

### Figure 2: Finite Size Scaling
Finite size scaling of the Luttinger parameter as a function of the system size $L$ according to the RG flow. The circles depict the results obtained from high precision DMRG calculations with open boundary conditions. Dashed lines show fits of the corresponding scaling form to the data. We show the three scaling regimes (i) in the superfluid phase according to $K_{\rm SF} \sim K^* + \kappa_0 \left(\frac{L_0}{L}\right)^{2(K^*-2)}$ (upper panel), (ii) for the value of $U$ closest to the separatrix using $K_{\rm sep} \sim 2  + \frac{1}{1/\kappa_0 + \ln L/L_0}$ (center panel), and (iii) in the insulating phase according to $K_{\rm MI} \sim 1 + (1+\kappa_0)\left(\frac{L_0}{L}\right)^{8 (1-\zeta)}$ (bottom panel).

In [ ]:
# requires to run the cell above first 
ζlookup = Dict{Float64,Float64}()
for (U,ζ) in zip(__Us,__ζs)
    ζlookup[U] = ζ
end 

In [ ]:
begin 
	local Us = [3.0,3.275,3.5]  
	local Ls = [16,32,48,64,96]
	local col_lookup = get_color_lookup(get_values_of_U(;path="../data/pbc/DMRG/"))
	local cols = [col_lookup[U] for U in Us]

	local xlim = (0.1,0.375)

	# mott insulator  
	local p3 = plot(;ylabel=L"$K$",xlabel=L"$1/\ln(L)$",xlim=xlim,yticks=([1.6,1.8,2.0],["1.60","1.80","2.00"]))
	local xoff=0.01
	annotate!(p3,0.12-xoff,1.89,text(latexstring(f"$U={Us[3]:2.1f}$"),10,:left)) 
	annotate!(p3,0.12352-xoff,2.0,text(latexstring(f"$\\zeta={ζlookup[Us[3]]:2.4f}$"),10,:left)) 
	# superfluid  
	local p1 = plot(;ylabel=L"$K$",xlim=xlim,xticks=(xticks(p3)[1][1],[""]),yticks=([2.36,2.37,2.38]),ylim=(2.35,2.39))
	annotate!(p1,0.120 -xoff,2.375,text(latexstring(f"$U={Us[1]:2.1f}$"),10,:left))
	annotate!(p1,0.12352-xoff,2.383,text(latexstring(f"$\\zeta={ζlookup[Us[1]]:2.4f}$"),10,:left)) 
	# ~sepraratrix 
	local p2 = plot(;ylabel=L"$K$",xlim=xlim,xticks=(xticks(p3)[1][1],[""]),yticks=([2.10,2.14,2.18]) )
	annotate!(p2,0.12-xoff,2.15,text(latexstring(f"$U={Us[2]:2.3f}$"),10,:left)) 
	annotate!(p2,0.12352-xoff,2.18,text(latexstring(f"$\\zeta={ζlookup[Us[2]]:2.4f}$"),10,:left)) 

	# x-axis transformation: plot 1/log(L)
	x_trafo = (x) -> 1.0 ./ log.(x)

	for (U, (i_c,c),pi) in zip(Us,enumerate(cols),(p1,p2,p3))
		local Ks_lin = zeros(Float64,length(Ls))
		local Ks_lin_err = zeros(Float64,length(Ls))
		local as_lin = zeros(Float64,length(Ls))
		for (i,L) in enumerate(Ls)
		    local ls, Fs = load_dmrg_pbc(L,U) 
		    local xs = lin_factor_pbc(ls,L)
		    local idx = window(L) 
			(Ks_lin[i], as_lin[i]), Ks_lin_err[i], _ = fit_fluc_lin_xs_pbc_err(xs,Fs,L,idx) 
		end   

		local Ls_fit = [5:1:1000;2000;5000;10000; 100000]
		if i_c == 2 
			# ~sepraratrix 
			local Ks = fss_separatrix(Ls_fit, (Ls[end],Ks_lin[end])) 
			plot!(pi, x_trafo(Ls_fit),Ks; linestyle=:dash, color=c)
		elseif i_c == 1
			# superfluid  
			local Ks = fss_superfluid(Ls_fit, (Ls[end-1:end], Ks_lin[end-1:end]))
			plot!(pi, x_trafo(Ls_fit),Ks; linestyle=:dash, color=c)
		elseif i_c == 3	
			# mott insulator  
			local Ks = fss_mott_insulator(Ls_fit, (Ls[end-1:end], Ks_lin[end-1:end]))
			plot!(pi, x_trafo(Ls_fit),Ks; linestyle=:dash, color=c)
		end 
		
		scatter!(pi, x_trafo(Ls), Ks_lin,yerr=Ks_lin_err; marker=:circle, msw=2, ms=5, nice_points(c)...)

	end
	
	# combine subplots
	local l = grid(3, 1)
	local boff = -7mm
	local p = plot(p1,p2,p3, layout = l, size=(1.2*300,1.2*250),bottom_margin=[boff boff 0mm])

	savefig(p,"../figures/002_finite_size_scaling.pdf");
	savefig(p,"../figures/002_finite_size_scaling.svg");
	#p
end 

### Figure 3: Obtaining Luttinger Parameter from Flucutations
Extracting the Luttinger parameter $K$ from the particle number fluctuations for different interaction strengths $U/J$ and systems of length $L=96$. We fit 

\begin{align*}
    \mathcal{F}_{\rm{pbc}}(\ell; K, \alpha) &= 
    \frac{K}{2\pi^2}\ln{\left[1+\frac{\sin^2{(\pi\ell/L)}}{\sinh^2{(\pi\alpha/L)}}\right]}   
\end{align*}

to the open boundary condition DMRG data for interval sizes close to $\ell=L/2$.

In [ ]:
begin  
	local Us = [3.000,3.275,3.5]


	local p1 = plot(;ylabel=L"$\mathcal{F}$",xlabel=L"$\frac{1}{2\pi^2}\,\ln\sin\frac{\pi \ell}{L}$", ) # legend(;pos=:bottomright)..., 
	  
	local col_lookup = get_color_lookup(get_values_of_U(;path="../data/pbc/DMRG/")) 
	local cols = [col_lookup[U] for U in Us]

	for (U, (i_c,c)) in zip(Us,enumerate(cols)) 

		local L = 96
		local ls, Fs = load_dmrg_pbc(L,U);
		local xs = lin_factor_pbc(ls, L);  
		local idx = window(L)
		local (Kfit, afit), Kerr ,  _ = fit_fluc_lin_xs_pbc_err(xs, Fs, L, idx)   
		 
		scatter!(p1, xs,Fs; marker=:circle, msw=2, ms=5, label=latexstring(f"\\ U={U:3.3f}"), nice_points(c)...)
		 
		plot!(p1, xs[5:end], fluctuations_pbc_large_L_xs(xs[5:end], (Kfit, afit), (L,)),color=:black,ls=:dash,label=nothing)    
		plot!(p1, xs[idx], fluctuations_pbc_large_L_xs(xs[idx], (Kfit, afit), (L,)),color=:black,label=nothing)   

		# legend:
		scatter!(p1,[-0.12],[0.62-0.09*(i_c-1)];ms=5,nice_points(c)...)
		annotate!(-0.12,0.62-0.09*(i_c-1),text(latexstring(f"\\ \\ \\ U={U:3.3f}"),:left,10))
  
	end
	local l = grid(1, 1)
	local p = plot(p1, layout = l, size=(1.2*300,1.2*200), bottom_margin=[1mm 0mm])

	savefig(p,"../figures/003_fit_K_to_fluctuations.pdf");
	savefig(p,"../figures/003_fit_K_to_fluctuations.svg");
	#p
end

#### Figure 4: Extracting Crtical Point
Dependence of the flow parameter $\zeta$ on the interaction strength $U/J$ (blue circles). At the critical point $\zeta=1$, and $\zeta(U)$ is linear close to the phase transition. A linear fit (orange line) reveals  the critical point $U_c/J=3.276\pm0.001$. The fit is performed to the region shown by the solid orange line and we extend the fit with a dashed line to show deviations away from $U_c$.

In [ ]:
begin  
	col_points = "#0C5DA5"
	col_fit = "#FFB347" 

	local p = plot(;xlabel=L"U/J",ylabel=L"\zeta",size=(1.2*315,1.2*225)) 
	# fit linear function to points close to Uc
	local lin_fun(U,(Uc,α),_) = α*(U .- Uc) .+ 1.0
	local mask = 5:7 
	local Uc, dUc, invUc, dinvUc, α, dα = py_fit_lin_err(__Us[mask],__ζs[mask],__ζserr[mask])  
	# plot data and fit
	scatter!(p,__Us,__ζs,yerr=__ζserr;ylim=(0.992,1.00999),xlim=(3.05,3.45),nice_points(col_points)...,ms=4)
	plot!(p,__Us[mask],lin_fun(__Us[mask],(Uc,α),());  color=col_fit ) 
	plot!(p,__Us,lin_fun(__Us,(Uc,α),()); ls=:dash, color=col_fit) 
	# put critial point as text
	text1 = latexstring(f"$U_c/J=\\,\\,\\, \\, \\,{Uc:4.3f}\\pm{dUc:3.3f}$")  
	text2 = latexstring(f"$J/U_c={invUc:4.4f}\\pm{dinvUc:3.4f}$") 
	annotate!(3.07,0.9945,text(text1,:left,:bottom,10))
	annotate!(3.07,0.9925,text(text2,:left,:bottom,10))
	

	savefig(p,"../figures/004_zeta_of_U.pdf");
	savefig(p,"../figures/004_zeta_of_U.svg");
	#p
end

## Supplementary Material

### Figure S1: Difference between open and periodic boundary conditions

In [ ]:
begin 
    local col_lookup = get_color_lookup(get_values_of_U(;path="../data/pbc/DMRG/")) 

    local p_pbc = plot(;axis=:off,size=(1.2*200,1.2*160),xlim=(-1.6,1.2),ylim=(-1.25,1.4), bottom_margin=[2mm 2mm] , top_margin=[2mm 2mm] )
    draw_periodic_lattice!(p_pbc ; n=13, nice_points("#000000")... , mswidth  =1 )
    
    local p_obc = plot(;axis=:off,size=(1.2*300,1.2*160), ylim=(-0.5,0.5),xlim=(-1,10.5))
    draw_open_lattice!(p_obc; nice_points("#000000")... , mswidth  =1 )
    p_obc  


    Us = [3.0,3.1,3.2,3.3]
    # fluctuations 
    window_fun_obc = L->round(Int,L/5):round(Int,L/3) 
    # pbc 
    local p_F_pbc = plot(;xlabel=L"$\frac{1}{2\pi^2}\,\ln\sin\frac{\pi \ell}{L}$",ylabel=L"$\mathcal{F}$ (periodic)",size=(1.2*315,1.2*225), bottom_margin=[2mm 2mm], xlim=(-0.2,0.02) ,ylim=(0.7,1.3))
    plot_fluctuations_supplement!(p_F_pbc, Us, 96, col_lookup; load_fun=load_dmrg_pbc)
    annotate!(p_F_pbc,-0.18,1.25,text(latexstring("L=96"),11,:left))

    # obc 
    local p_F_obc = plot(;xlabel=L"$\frac{1}{2\pi^2}\,\ln\sin\frac{\pi \ell}{L}$",ylabel=L"$\mathcal{F}$ (open)",size=(1.2*315,1.2*225), bottom_margin=[2mm 2mm] , xlim=(-0.2,0.02) ,ylim=(1.05,1.65))
    plot_fluctuations_supplement!(p_F_obc, Us, 460, col_lookup; load_fun=load_dmrg_obc, legend=false,window_fun=window_fun_obc, first_point=22,first_fit_value=30)
    annotate!(p_F_obc,-0.18,1.6,text(latexstring("L=460"),11,:left))

    
    # K(L)
    x_trafo = (x) -> 1.0 ./ log.(x)
    # pbc 
    local p_K_pbc = plot(;xlabel=L"$1/\ln(L)$",ylabel=L"$K$ (periodic)",size=(1.2*315,1.2*225), bottom_margin=[2mm 2mm] , xlim=(0.17,0.4))
    plot_K_supplement!(p_K_pbc, Us, col_lookup; load_fun=load_dmrg_pbc, L_lookup=U->get_values_of_L("../data/pbc/DMRG/",U) )
    # obc 
    local p_K_obc = plot(;xlabel=L"$1/\ln(L)$",ylabel=L"$K$ (open)", size=(1.2*315,1.2*225), bottom_margin=[2mm 2mm],xlim=(0.14,0.22))
    plot_K_supplement!(p_K_obc, Us, col_lookup; load_fun=load_dmrg_obc ,L_lookup=U->get_values_of_L_obc("../data/obc/DMRG/",U),window_fun=window_fun_obc,mask_err=(xs)->length(xs)÷4:2length(xs)÷3)

    # combine plots 
    local l = grid(3, 2; heights=[0.15,0.425,0.425], widths=[0.5,0.5 ])

    local p = plot(p_pbc,p_obc, p_F_pbc, p_F_obc, p_K_pbc, p_K_obc,layout=l, size=2.0.*(1.0*315,1.0*325), bottom_margin=[-10mm -10mm 2mm 2mm 2mm 2mm], top_margin = [0mm 0mm 0mm 0mm 0mm 0mm], left_margin=[2mm 2mm])

    savefig(p,"../figures/S001_compare_pbc_obc.pdf");
    savefig(p,"../figures/S001_compare_pbc_obc.svg");
end